-csv datei einlesen
-Name ändern
- 1sten 5 zeilen ausgeben

In [46]:
import pandas as pd
from datetime import datetime, timedelta
csv_file = '../Data_Cleanup/outCSV/Clean_Metro_Set.csv'
metro = pd.read_csv(csv_file)

C:\Users\moham\AppData\Local\Temp\ipykernel_12584\399612730.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metro = pd.read_csv(csv_file)


   Unnamed: 0.1  Unnamed: 0           start_time             end_time  \
0             0           0  2018-05-29 16:45:00  2018-05-29 17:08:00   
1             1         891  2019-08-16 17:12:00  2019-08-16 17:42:00   
2             2        1467  2019-01-30 12:22:00  2019-01-30 12:28:00   
3             3        6956  2018-05-15 08:23:00  2018-05-15 08:30:00   
4             4       11733  2018-11-03 18:30:00  2018-11-03 18:51:00   

   start_station_id  start_station_lat  start_station_lon  end_station_id  \
0              3068            34.0532        -118.250954            3014   
1              3068            34.0532        -118.250954            3014   
2              3068            34.0532        -118.250954            3014   
3              3068            34.0532        -118.250954            3014   
4              3068            34.0532        -118.250954            3014   

   end_station_lat  end_station_lon bike_id  
0         34.05661      -118.237213    5772  
1     

-Tabelle sortieren nach Transaktionszeit

In [47]:
metro['start_time'] = pd.to_datetime(metro['start_time'])

metro ['end_time'] = pd.to_datetime(metro['end_time'])

metro = metro.sort_values(by='start_time')

-Neue Spalte hinzufügen für Idle_Time
-Alle neuen felder mit "None" initialisieren, da alle nicht belegte Idle Time Zellen keinen Einfluss später in der durchschnittlichen Idle Time Berechnung haben sollten.


In [48]:
metro['idle_time'] = None

Idle Time berechnung

Erklärung des folgenden Codes:

Die Berechnung der Idle Time basiert auf der Überprüfung der Startzeit (current_start_time) im Vergleich zur Endzeit (stored end time) des vorherigen Vorgangs mit der gleichen Startstation (current_start_station_id). 

Der Algorithmus durchläuft die Datenzeilen des DataFrames und speichert die Endzeiten (current_end_time) für jede Endstation (current_end_station_id) in einem Dictionary namens BigDict. 

Für jede Zeile wird überprüft, ob die aktuelle Startzeit größer ist als die gespeicherte Endzeit für die Startstation. Wenn dies der Fall ist, bedeutet dies, dass zwischen den Vorgängen mit der gleichen Startstation eine Leerlaufzeit aufgetreten ist. Die Leerlaufzeit wird berechnet, indem die Startzeit von der gespeicherten Endzeit subtrahiert wird. Das Ergebnis wird dann in der Spalte "idle_time" der entsprechenden Zeile des DataFrames gespeichert.

Um die Endzeit für eine Startstation abzurufen, wird das Dictionary BigDict verwendet. Wenn eine Startstation zum ersten Mal auftritt und keine gespeicherte Endzeit vorhanden ist, wird ein KeyError ausgelöst. In diesem Fall wird die aktuelle Endzeit für die aktuelle Endstation dem BigDict hinzugefügt.

Durch dieses Verfahren wird für jede Zeile im DataFrame die Leerlaufzeit berechnet, sofern eine vorherige Operation mit derselben Startstation vorhanden ist. Dadurch können Informationen über Leerlaufzeiten zwischen aufeinanderfolgenden Vorgängen mit derselben Startstation erfasst und im DataFrame gespeichert werden.

In [49]:
BigDict = {}  # Initialize an empty dictionary to store station IDs and end times

# Iterate through each row in the DataFrame
for i, row in metro.iterrows():
    current_start_station_id = metro.loc[i, "start_station_id"]
    current_end_station_id = metro.loc[i, "end_station_id"]
    current_start_time = metro.loc[i, "start_time"]
    current_end_time = metro.loc[i, "end_time"]

    try:
        # Check if current start time is greater than the stored end time for the start station ID
        if current_start_time > BigDict[str(current_start_station_id)]:
            # Calculate idle time as the difference between the current start time and the stored end time
            metro.at[i, "idle_time"] = current_start_time - BigDict[str(current_start_station_id)]
    except KeyError:
        # If there is no stored end time for the start station ID, handle the KeyError exception
        BigDict[str(current_end_station_id)] = current_end_time

    # Store the current end time for the end station ID in BigDict
    BigDict[str(current_end_station_id)] = current_end_time

In [50]:
metro.to_csv('../Data_Cleanup/outCSV/Metro_Set_with_IdleTime.csv')